In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [103]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=0, bias=False) #28x28x1 -> 26x26x8, RF:3
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=0, bias=False) # 26x26x8 -> 24x24x8, RF:5
        self.bn2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=0, bias=False) # 24x24x8 -> 22x22x8, RF:7
        self.bn3 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2) # 11x11x8, RF:8
        self.conv4 = nn.Conv2d(8, 16, 3, padding=0, bias=False) # 11x11x8 -> 9x9x16, RF:12  
        self.bn4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 9x9x16 -> 7x7x16, RF:16 
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 7x7x16 -> 5x5x16, RF:20
        self.bn6 = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 5x5x16- > 3x3x16, RF:24
        self.bn7 = nn.BatchNorm2d(16)
        self.conv8 = nn.Conv2d(16, 32, 3, padding=0, bias=False) # 3x3x16- > 1x1x16, RF:28
        #self.gap = nn.AvgPool2d(3) # 3x3x16 -> 1x1x16
        self.op = nn.Conv2d(32, 10, 1, 1, padding=0, bias=False)
        
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.bn3(F.relu(self.conv3(x)))
        
        x = self.pool1(x)
        
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.bn5(F.relu(self.conv5(x)))
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.bn7(F.relu(self.conv7(x)))
        
        x = F.relu(self.conv8(x))
        #x = self.gap(x)
        x = self.op(x)
        
        x = x.squeeze(dim=(-2,-1))        
        return F.log_softmax(x, dim=-1)
        
        

In [118]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False) #28x28x1 -> 28x28x8, RF:3
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1, bias=False) # 28x28x8 -> 28x28x8, RF:5
        self.bn2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1, bias=False) # 28x28x8 -> 28x28x8, RF:7
        self.bn3 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2) # 14x14x8, RF:8
        self.dp = nn.Dropout(p=0.2)
        self.conv4 = nn.Conv2d(8, 16, 3, padding=0, bias=False) # 14x14x8 -> 12x12x16, RF:12  
        self.bn4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 12x12x16 -> 10x10x16, RF:16 
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 10x10x16 -> 8x8x16, RF:20
        self.bn6 = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # 8x8x16- > 6x6x16, RF:24
        self.bn7 = nn.BatchNorm2d(16)
        self.conv8 = nn.Conv2d(16, 32, 3, padding=0, bias=False) # 6x6x16- > 4x4x16, RF:28
        self.gap = nn.AvgPool2d(4) # 4x4x16 -> 1x1x16
        self.op = nn.Conv2d(32, 10, 1, 1, padding=0, bias=False)
        
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.bn3(F.relu(self.conv3(x)))
        
        x = self.pool1(x)
        x = self.dp(x)
        
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.bn5(F.relu(self.conv5(x)))
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.bn7(F.relu(self.conv7(x)))
        
        x = F.relu(self.conv8(x))
        x = self.gap(x)
        x = self.op(x)
        
        x = x.squeeze(dim=(-2,-1))        
        return F.log_softmax(x, dim=-1)
        
        

In [119]:
model = Net().to(device)
xx = torch.randn(1,1,28,28).to(device)
yy = model(xx)
yy.squeeze(dim=(-1,-2)).shape

torch.Size([10])

In [120]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             576
       BatchNorm2d-4            [-1, 8, 28, 28]              16
            Conv2d-5            [-1, 8, 28, 28]             576
       BatchNorm2d-6            [-1, 8, 28, 28]              16
         MaxPool2d-7            [-1, 8, 14, 14]               0
           Dropout-8            [-1, 8, 14, 14]               0
            Conv2d-9           [-1, 16, 12, 12]           1,152
      BatchNorm2d-10           [-1, 16, 12, 12]              32
           Conv2d-11           [-1, 16, 10, 10]           2,304
      BatchNorm2d-12           [-1, 16, 10, 10]              32
           Conv2d-13             [-1, 16, 8, 8]           2,304
      BatchNorm2d-14             [-1, 1

In [121]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [122]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [123]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0602, Accuracy: 9813/10000 (98.130%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0377, Accuracy: 9876/10000 (98.760%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0191, Accuracy: 9941/10000 (99.410%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0198, Accuracy: 9938/10000 (99.380%)



loss=0.022418051958084106 batch_id=468: 100%|████████████████████████████████████████| 469/469 [00:07<00:00, 59.54it/s]



Test set: Average loss: 0.0197, Accuracy: 9936/10000 (99.360%)

